# Conditional tasks in AI agents

The following code suppresses warning messages to ensure the output is clean and focused on the essential results.

In [2]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'

Imports essential modules from CrewAI. Also, sets up environment variables to securely store API keys required for accessing external services.

In [17]:
from crewai import Agent, Task, Crew, Process

import os
os.environ["OPENAI_API_KEY"] = 'ADD YOUR API KEY HERE'
os.environ["SERPER_API_KEY"] = 'ADD YOUR API KEY HERE'

Sets up the conditional task checker

In [11]:
from typing import List
from pydantic import BaseModel
from crewai.tasks.conditional_task import ConditionalTask
from crewai.tasks.task_output import TaskOutput
from crewai_tools import SerperDevTool

# Define a function to assess whether the data needs to be augmented
def should_fetch_more_data(output: TaskOutput) -> bool:
    return len(output.pydantic.events) < 8  # Condition to trigger task

Agents declared

In [12]:
# Create agents for different roles
data_collector = Agent(
    role="Data Collector",
    goal="Retrieve event data using Serper tool",
    backstory="You have a knack for finding the most exciting events happening around.",
    verbose=True,
    tools=[SerperDevTool()],
)

data_analyzer = Agent(
    role="Data Analyzer",
    goal="Analyze the collected data",
    backstory="You're known for your analytical skills, making sense of complex datasets.",
    verbose=True,
)

summary_creator = Agent(
    role="Summary Creator",
    goal="Produce a concise summary from the event data",
    backstory="You're a skilled writer, able to summarize information clearly and effectively.",
    verbose=True,
)

Tasks are declared as we did before however the conditional task for this example is declared with `ConditionalTask()` instead

In [13]:
class EventsData(BaseModel):
    events: List[str]

# Define the tasks
fetch_task = Task(
    description="Collect event data for New York City using Serper tool",
    expected_output="A list of 8 exciting events happening in NYC this week",
    agent=data_collector,
    output_pydantic=EventsData,
)

verify_data_task = ConditionalTask(
    description="""
        Ensure that sufficient event data has been collected.
        If fewer than 8 events are found, gather more using the Serper tool.
        """,
    expected_output="An updated list of at least 8 events happening in NYC this week",
    condition=should_fetch_more_data,
    agent=data_analyzer,
)

summary_task = Task(
    description="Summarize the collected events data for NYC",
    expected_output="summary_generated",
    agent=summary_creator,
)

In [15]:
# Assemble the crew with the defined agents and tasks
crew = Crew(
    agents=[data_collector, data_analyzer, summary_creator],
    tasks=[fetch_task, verify_data_task, summary_task],
    verbose=True,
    planning=True  # Retain the planning feature
)

# Execute the tasks with the crew
result = crew.kickoff()
print("results", result)

 [2024-09-02 06:41:29][INFO]: Planning the crew execution
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
 [2024-09-02 06:41:43][DEBUG]: == Working Agent: Data Collector
 [2024-09-02 06:41:43][INFO]: == Starting Task: Collect event data for New York City using Serper toolTask Number 1: Collect Event Data for New York City

Agent: Data Collector
Agent Goal: Retrieve event data using the Serper tool
Tools Available: SerperDevTool

1. Initiate the Serper Tool:
   - Open the Serper tool interface.
   - Ensure the tool is ready to accept a search query.

2. Craft a Search Query:
   - Formulate a search query that specifies the request for events in New York City. An example query could be: "exciting events happening in New York City this week".

3. Execute the Search:
   - Input the crafted search query into the Serper tool.
   - Click the search button to execute the query.

4. Review the Search Results:
   - Once the results are returned,

In [16]:
from IPython.display import Markdown

# Convert the CrewOutput object to a Markdown string
result_markdown = result.raw

# Display the result as Markdown
Markdown(result_markdown)

Here are 8 exciting events happening in New York City this week:

1. **Event Name**: Baile De Favela Rooftop Party • Carnaval & Micareta
   - **Date**: Saturday, September 7, 2024
   - **Location**: The DL, New York, NY
   - **Description**: A vibrant rooftop party celebrating Brazilian Carnaval and Micareta with music and dancing. Features one-of-a-kind performances, carioca brindes, catuaba shots, open de coxinha, dance contests, and more.
   - **Link**: [Eventbrite](https://www.eventbrite.com/e/baile-de-favela-rooftop-party-carnaval-micareta-tickets-974273948607)

2. **Event Name**: Flag Fest Labor Day Sunday
   - **Date**: Sunday, September 1, 2024
   - **Location**: SOB's, 204 Varick St, New York, NY
   - **Description**: A colorful and lively celebration for Labor Day featuring music, dancing, and festivities. Playing Dancehall, Soca, Afrobeats, Reggae, and more. Everyone free before 11:30pm with RSVP.
   - **Link**: [Eventbrite](https://www.eventbrite.com/e/flag-fest-labor-day-sunday-sobs-dancehall-soca-more-tickets-870944226627)

3. **Event Name**: Labor Day Weekend Rooftop Shutdown Day Party
   - **Date**: Sunday, September 1, 2024
   - **Location**: The Delancey, 168 Delancey St, New York, NY
   - **Description**: A rooftop day party to celebrate Labor Day weekend with music, drinks, and a great atmosphere. Everyone Free Entry with RSVP + Food Menu available. Doors open from 3pm-9pm.
   - **Link**: [Eventbrite](https://www.eventbrite.com/e/labor-day-weekend-rooftop-shutdown-day-party-the-delancey-tickets-853579618617)

4. **Event Name**: Glow Party
   - **Date**: This weekend
   - **Location**: The Delancey, New York, NY
   - **Description**: An exciting glow-in-the-dark party with neon lights, music, and dancing. Features live music, foam sticks, glow sticks, LED glasses, zebras, and more.
   - **Link**: [Facebook](https://m.facebook.com/events/948797505322642/?acontext=%7B%22ref%22%3A%2244%22%2C%22action_history%22%3A%22%5B%7B%5C%22surface%5C%22%3A%5C%22permalink%5C%22%2C%5C%22mechanism%5C%22%3A%5C%22RHC%5C%22%2C%5C%22extra_data%5C%22%3A%5B%5D%7D%5D%22%7D)

5. **Event Name**: A:M Jamboree NYC
   - **Date**: Monday, September 2, 2024
   - **Location**: Coney Art Walls, Brooklyn, NY
   - **Description**: A lively jamboree event featuring music, dancing, and entertainment. Includes signature A:M Party Mug, guest artiste performances, and captivating views with multiple picture backdrops.
   - **Link**: [Eventbrite](https://www.eventbrite.com/e/am-jamboree-nyc-tickets-936214060457)

6. **Event Name**: Matt Haig celebrates THE LIFE IMPOSSIBLE at B&N Union Square
   - **Date**: Tuesday, September 3, 2024
   - **Location**: B&N Union Square, New York, NY
   - **Description**: Author Matt Haig celebrates his new book "THE LIFE IMPOSSIBLE" with a reading and signing event. The event starts at 6PM.
   - **Link**: [Eventbrite](https://www.eventbrite.com/e/matt-haig-celebrates-the-life-impossible-at-bn-union-square-tickets-915420205477)

7. **Event Name**: Two Dykes and a Mic: Topping Your Best Friend Tour
   - **Date**: Saturday, September 7, 2024, at 7:30 PM
   - **Location**: The Bell House, Brooklyn, NY
   - **Description**: A comedy tour featuring the hilarious duo "Two Dykes and a Mic". Dedicated to sex positivity, queer dating, and building a community filled with laughter and gayness.
   - **Link**: [Eventbrite](https://www.eventbrite.com/e/two-dykes-and-a-mic-topping-your-best-friend-tour-tickets-939974317487)

8. **Event Name**: MIDNIGHTS - A TAYLOR SWIFT DANCE PARTY
   - **Date**: Saturday, September 7, 2024, at 11:00 PM
   - **Location**: DROM, New York, NY
   - **Description**: A dance party dedicated to Taylor Swift fans, featuring music from her latest album "Midnights". Hosted by DJ Alex Jay. This is a 21+ event.
   - **Link**: [Eventbrite](https://www.eventbrite.com/e/midnights-a-taylor-swift-dance-party-tickets-876094792107)

Enjoy your week in New York City!